In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Avi').getOrCreate()

23/03/07 09:10:41 WARN Utils: Your hostname, avijit-HP-Laptop-15q-bu0xx resolves to a loopback address: 127.0.1.1; using 192.168.18.7 instead (on interface wlo1)
23/03/07 09:10:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/07 09:10:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
retail_df = spark.read.format('csv').option('inferSchema',"True").option("header",'true').load("./data/retail_all.csv")

In [4]:
retail_df.show(2) 

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/01/10 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/01/10 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 2 rows



In [7]:
retail_df.count() # Here Count is used as an action

541909

# Count() as a transformation

In [8]:
from pyspark.sql.functions import count
retail_df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



when performing count(*) spark will count all null values, however when counting an individual column spark will not count the null values 

# Number of distinct groups

In [10]:
from pyspark.sql.functions import sumDistinct
retail_df.select(sumDistinct("Quantity")).show()

/home/avijit/.local/lib/python3.8/site-packages/pyspark/sql/functions.py:316: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



# avg

In [11]:
from pyspark.sql.functions import sum, count, avg, expr

In [12]:
retail_df.select(
count('Quantity').alias("total_transanctions"),
sum("Quantity").alias("total_purchases"),
avg("Quantity").alias("avg_purchases"),
expr("mean(Quantity)").alias("mean_purchases")
).selectExpr(
    "total_purchases/total_transanctions",
    "avg_purchases",
    "mean_purchases"
    
).show()

+---------------------------------------+----------------+----------------+
|(total_purchases / total_transanctions)|   avg_purchases|  mean_purchases|
+---------------------------------------+----------------+----------------+
|                       9.55224954743324|9.55224954743324|9.55224954743324|
+---------------------------------------+----------------+----------------+



# Aggregating to Complex types

In [18]:
from pyspark.sql.functions import collect_set,collect_list
a_= retail_df.agg(collect_set("Country"),collect_list("Country")).collect()

In [27]:
len(a_[0][1])

541909

# Grouping 

In [30]:
retail_df.groupBy("InvoiceNo","CustomerId").count().show(2)

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
+---------+----------+-----+
only showing top 2 rows



# grouping with maps

In [31]:
retail_df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1